In [1]:
# !pip install geojson
# !pip install folium
# !pip install geopandas
# !pip install pyogrio

In [2]:
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import json
import folium
import geopandas as gpd
import pyogrio
from shapely.ops import split
from shapely.geometry import MultiLineString, LineString, Point, Polygon, MultiPolygon
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster
import random
import requests
import json
from urllib.parse import urlencode

In [3]:
from utils import get_distance, get_isochrone, get_random_point_in_geometry, get_routing

### DB IGN : Routes, Points d'intérêts, Habitations

### Loading data for La Baule

In [4]:
file_ign_44 = '../Data/BDCARTO/44_Loire_Atlantique/1_DONNEES_LIVRAISON_2024-12-00080/data.gpkg'
gdf_commune = gpd.read_file(file_ign_44, layer="commune")
gdf_la_baule = gdf_commune[gdf_commune['code_postal'] == '44500']

gdf_la_baule_gps = gdf_la_baule.to_crs(epsg=4326)
gdf_la_baule_gps = gdf_la_baule_gps.drop(columns=['date_du_recensement'])
geometry_la_baule_gps = gdf_la_baule_gps.iloc[0].geometry
center_lat, center_lon = geometry_la_baule_gps.centroid.y, geometry_la_baule_gps.centroid.x

# Routing

In [5]:
# def get_random_point_in_geometry(geometry):
#     """Generate one random point inside the given geometry 
#        and return a GeoDataFrame with its coordinates."""
    
#     min_x, min_y, max_x, max_y = geometry.bounds
#     while True:
#         random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
#         if geometry.contains(random_point):
#             # Create a GeoDataFrame with the point
#             gdf = gpd.GeoDataFrame(pd.DataFrame({'x': [random_point.x], 'y': [random_point.y]}),
#                                    geometry=[random_point], crs="EPSG:4326")
#             return gdf

## Routing between two points

In [6]:
# Generate two random points inside the selected geometry
df_start_point = get_random_point_in_geometry(geometry_la_baule_gps)
df_end_point = get_random_point_in_geometry(geometry_la_baule_gps)

start_point = [df_start_point.iloc[0].geometry.centroid.x, df_start_point.iloc[0].geometry.centroid.y]
end_point = [df_end_point.iloc[0].geometry.centroid.x, df_end_point.iloc[0].geometry.centroid.y]

# Centrer la carte sur La Baule (approx.)
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Ajouter le contour de la ville
folium.GeoJson(
    gdf_la_baule_gps,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

folium.GeoJson(
    df_start_point,
    name="Path between points",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2}
).add_to(m)

folium.GeoJson(
    df_end_point,
    name="Path between points",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2}
).add_to(m)

# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m

In [ ]:
data = get_routing(start_point, end_point)
data.keys()

dict_keys(['resource', 'resourceVersion', 'start', 'end', 'profile', 'optimization', 'geometry', 'crs', 'distanceUnit', 'timeUnit', 'bbox', 'distance', 'duration', 'constraints', 'portions'])

In [8]:
# Extract important information
distance = data.get("distance")
distance_unit = data.get("distanceUnit")
duration = data.get("duration")
time_unit = data.get("timeUnit")

# Print the formatted information
print(f"Distance: {distance} {distance_unit}")
print(f"Duration: {duration} {time_unit}")

Distance: 14.8898 kilometer
Duration: 19.196666666666665 minute


In [9]:
# Convert coordinates to a LineString (longitude, latitude format)
line_geom = LineString(data['geometry']["coordinates"])

# Create a GeoDataFrame
gdf_routing_gps = gpd.GeoDataFrame([{"geometry": line_geom}], crs="EPSG:4326")  # WGS 84 CRS

# Display the GeoDataFrame
gdf_routing_gps

,geometry
0,"LINESTRING (-2.33029 47.27751, -2.33015 47.277..."


In [10]:
# Centrer la carte sur La Baule (approx.)
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Ajouter le contour de la ville
folium.GeoJson(
    gdf_la_baule_gps,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Ajouter le chemin entre les points 
folium.GeoJson(
    gdf_routing_gps,
    name="Path between points",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2}
).add_to(m)

folium.GeoJson(
    df_start_point,
    name="Path between points",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2}
).add_to(m)

folium.GeoJson(
    df_end_point,
    name="Path between points",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2}
).add_to(m)

# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m

## Reachable area from one point

In [12]:
# Example: Get an isochrone for a 5-minute drive (300 seconds)
# point_coords = (2.337306, 48.849319)  # Longitude, Latitude
point_coords = start_point

# Example constraints (ban highways)
constraints_example = {
    "constraintType": "banned",
    "key": "wayType",
    "operator": "=",
    "value": "autoroute"
}

cost_value = 300
cost_type = 'time'

# Call the function
isochrone_data = get_isochrone(point_coords, cost_value=cost_value, cost_type=cost_type, constraints=constraints_example)

In [13]:
# Convert coordinates to a Shapely Polygon
polygon = Polygon(isochrone_data['geometry']["coordinates"][0])  # Extract first polygon ring

# Create a GeoDataFrame
gdf_routing_area = gpd.GeoDataFrame({'geometry': [polygon]}, crs="EPSG:4326")  # WGS84

In [14]:
# Centrer la carte sur La Baule (approx.)
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Ajouter le contour de la ville
folium.GeoJson(
    geometry_la_baule_gps,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Ajouter la zone reachable à partir d'un point  
folium.GeoJson(
    gdf_routing_area,
    name="Reachable area",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2},
    tooltip=f"Reachable area within {cost_value} {cost_type}"
).add_to(m)

# Point de départ
folium.GeoJson(
    df_start_point,
    name="Reachable area",
    style_function=lambda x: {"color": "green", "weight": 4, "alpha": 0.2},
    tooltip=f"Starting point"
).add_to(m)

# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m

# Routing between centroids of IRIS zones

In [15]:
import geopandas as gpd
import pandas as pd

In [16]:
gdf_iris_contours = gpd.read_file('../data/IRIS/contours-iris.gpkg')
gdf_iris_contours['code_insee'] = pd.to_numeric(gdf_iris_contours['code_insee'], errors='coerce')
gdf_iris_contours = gdf_iris_contours.dropna(subset=['code_insee'])
gdf_iris_contours['code_insee'] = gdf_iris_contours['code_insee'].astype(int)
gdf_baule_iris = gdf_iris_contours[gdf_iris_contours['code_insee'] ==  44055]
gdf_baule_iris['centroid'] = gdf_baule_iris.geometry.centroid

# GPS format
gdf_baule_iris_gps = gdf_baule_iris.to_crs("EPSG:4326")
gdf_baule_iris_gps['centroid']  = gdf_baule_iris_gps['centroid'].to_crs("EPSG:4326")

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001E4E1EDDF70>>
Traceback (most recent call last):
  File "C:\Users\eliot\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\eliot\anaconda3\Lib\threading.py", line 1533, in enumerate
    def enumerate():
    
KeyboardInterrupt: 
c:\Users\eliot\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
# def get_distance(start_point, end_point):
#     routing = get_routing(start_point, end_point)
#     distance = routing.get("distance")
#     return distance

In [ ]:
# Create an empty DataFrame to store distances
distances_df = pd.DataFrame(index=gdf_baule_iris_gps.index, columns=[f"dist_to_{i}" for i in gdf_baule_iris_gps.index])

# Compute distances
for i, row_i in gdf_baule_iris_gps.iterrows():
    for j, row_j in gdf_baule_iris_gps.iterrows():
        distances_df.at[i, f"dist_to_{j}"] = get_distance(row_i["centroid"].coords[0], row_j["centroid"].coords[0])

# Merge the distance DataFrame back into the GeoDataFrame
gdf_baule_iris_gps = gdf_baule_iris_gps.join(distances_df)

Generated URL: https://data.geopf.fr/navigation/itineraire?resource=bdtopo-osrm&start=-2.367824679256029%2C47.280523183432805&end=-2.367824679256029%2C47.280523183432805&profile=car&optimization=fastest&constraints=%7B%22constraintType%22%3A%22banned%22%2C%22key%22%3A%22wayType%22%2C%22operator%22%3A%22%3D%22%2C%22value%22%3A%22autoroute%22%7D&getSteps=true&getBbox=true&distanceUnit=kilometer&timeUnit=minute&crs=EPSG%3A4326
Response: {'resource': 'bdtopo-osrm', 'resourceVersion': '2025-01-22', 'start': '-2.367616,47.2807', 'end': '-2.367616,47.2807', 'profile': 'car', 'optimization': 'fastest', 'geometry': {'coordinates': [[-2.367616, 47.2807], [-2.367616, 47.2807]], 'type': 'LineString'}, 'crs': 'EPSG:4326', 'distanceUnit': 'kilometer', 'timeUnit': 'minute', 'bbox': [-2.367616, 47.2807, -2.367616, 47.2807], 'distance': 0, 'duration': 0, 'constraints': [{'type': 'banned', 'key': 'waytype', 'operator': '=', 'value': 'autoroute'}], 'portions': [{'start': '-2.367616,47.2807', 'end': '-2.3

In [ ]:
gdf_baule_iris_gps.head()

,cleabs,code_insee,nom_commune,iris,code_iris,nom_iris,type_iris,geometry,centroid,dist_to_1851,dist_to_3039,dist_to_4924,dist_to_5788,dist_to_6709,dist_to_7305
1851,IRIS____0000000440550101,44055,La Baule-Escoublac,0101,440550101,La Baule les Pins,H,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",POINT (-2.36782 47.28052),0,2.8856,3.3062,2.3819,2.9707,4.3403
3039,IRIS____0000000440550102,44055,La Baule-Escoublac,0102,440550102,Centre-Benoît,H,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",POINT (-2.40124 47.2817),3.013,0,0.7609,3.371,7.4203,11.3524
4924,IRIS____0000000440550103,44055,La Baule-Escoublac,0103,440550103,Gare-Grand Clos,H,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",POINT (-2.4036 47.28485),3.2267,0.5556,0,3.0541,7.0775,11.0097
5788,IRIS____0000000440550104,44055,La Baule-Escoublac,0104,440550104,Beslon,H,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",POINT (-2.38227 47.29256),2.3327,3.0183,3.0211,0,2.6009,5.0269
6709,IRIS____0000000440550105,44055,La Baule-Escoublac,0105,440550105,Escoublac,H,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",POINT (-2.35904 47.30102),3.0162,4.8215,4.8243,2.4543,0,4.1328


In [ ]:
gdf_baule_iris_gps = gdf_baule_iris_gps.drop(columns=["centroid"])
gdf_baule_iris_gps.to_file("iris_centroids_distance.geojson", driver="GeoJSON")

In [ ]:
gdf_baule_iris_gps.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich